In [2]:
%pip install datasets
%pip install jsonlines

  Using cached filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/536.6 kB ? eta -:--:--
   ---- ----------------------------------- 61.4/536.6 kB 1.1 MB/s eta 0:00:01
   --------- ------------------------------ 122.9/536.6 kB 1.2 MB/s eta 0:00:01
   ---------- ----------------------------- 143.4/536.6 kB 1.2 MB/s eta 0:00:01
   ---------- ----------------------------- 143.4/536.6 kB 1.2 MB/s eta 0:00:01
   ----------- -------------------------- 163.8/536.6 kB 701.4 kB/s eta 0:00:01
   -------------- ----------------------- 204.8/536.6 kB 689.9 kB/s eta 0:00:01
   ------------------ ------------------- 256.0/536.6 kB 749.3 kB/s eta 0:00:01
   ----------------------- -------------- 337.9/536.6 kB 873.8 kB/s eta 0:00:01
   --------------------------- ---------- 389.1/536.6 kB 930.9 kB/s eta 0:00:01
   ------------------------------------ --- 491.5/536.6 kB 1.0 MB/s eta 0:00:

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Import del dataset

In [2]:
import json
import pandas as pd
import os

data_path = '../data/'
dataset_path = os.path.join(os.getcwd(), data_path + 'annotations.jsonl')

data = []

with open(dataset_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))


Prendere colonne e metterle in variabili separate

In [3]:
df = pd.DataFrame(data)
id_tmp = df['id'].astype(str)
text_tmp = df['text'].astype(str)
label_tmp = df['label'].astype(str)
df.drop(['Comments'], axis=1)

,id,text,label
0,1,La giornata spuntÃ² serena e limpida per gli s...,"[[44, 49, PER], [87, 91, GPE], [132, 142, GPE]..."
1,2,I .\nLa ditta portava il nome del padre Giovan...,"[[7, 12, ORG], [33, 58, PER], [83, 107, PER], ..."
2,3,I .\nLa prima volta che Cesare Lascaris entrÃ²...,"[[23, 38, PER], [48, 70, FAC], [59, 70, PER], ..."
3,4,"I .\nDinanzi all ' osteria di Muzio Scevola , ...","[[18, 42, FAC], [48, 58, LOC], [139, 153, GPE]..."
4,5,"Anna cosÃ¬ racconta : Per lungo tempo , l ' am...","[[0, 4, PER], [64, 80, PER], [151, 179, PER], ..."
...,...,...,...
95,96,"I .\nEllade , giovinezza del mondo .\nNel temp...","[[4, 10, GPE], [28, 33, LOC], [72, 78, PER], [..."
96,97,CAPITOLO PRIMO .\nNel quale Phileas Fogg e Gam...,"[[27, 39, PER], [42, 52, PER], [85, 88, PER], ..."
97,98,"Clotilde entrÃ² un poco sbadatamente , cantand...","[[0, 8, PER], [53, 83, FAC], [75, 83, FAC], [1..."
98,99,"I .\nSenza fiori nascosti nella sottoveste , m...","[[125, 138, PER], [200, 207, PER], [366, 372, ..."


Prendere da text parola che inizia e finisce con indice di label

In [4]:
print(df.columns)

Index(['id', 'text', 'label', 'Comments'], dtype='object')


In [5]:
import pandas as pd
import re
import jsonlines


pattern = r'\[(\d+), (\d+), \'(\w+)\'\]'
lst = []

source = []
targets = []

output_path = os.path.join(os.getcwd(), data_path + 'datasetT5.jsonl')

with jsonlines.open(output_path, 'w') as writer:

    for index, sub in enumerate(label_tmp):    
        lst = re.findall(pattern, str(sub))
        source = text_tmp[index]
        for item in lst:
            text_tmp[index] = text_tmp[index].replace('\"','"')
            text_tmp_substring = text_tmp[index][int(item[0]) + 1  :int(item[1]) +1 ]
            if(text_tmp_substring.startswith(" ")):
                text_tmp_substring = text_tmp[index][int(item[0]) + 2  :int(item[1]) + 2 ]

            targets.append(text_tmp_substring + " [" + item[2] + "]")
            
        writer.write({'source': source, 'targets': targets})
        
        targets = []
                